## Welcome to the temperature trends notebook! 
     
#### **Audience:** Anybody with a computer and access to at least 4GB of memory.
#### **Intent:** Build familiarity with NOAA's Climate at a Glance dataset and the analysis of climate trends.           
#### **Outcome:** Statistics, graphs, and plots of temperature trends.           
This is a Jupyter Notebook meant to facilitate the analysis of temperature data over time, using historical records stored at NOAA's National Centers for Environmental Information (NCEI). Users will download and analyze real scientific data and see the temperature trends in their desired region of the United States or the globe. Because users will be using real local data, some data records may be incomplete. As such, it is important to use scientific analysis skills to assess the usability of the chosen dataset and resulting analyses. 

#### **Read and follow these steps below before beginning the notebook.**     
1. **Go to https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/ and:**
    1. Click on the tab of your desired region (i.e. Nation, State, Division, etc.), then launch "Time Series"
    2. Choose your temperature Parameter (maximum temperature, minimum temperature, or average temperature).
    3. Choose a Time Scale of 12-month. 
    4. Choose the month of December, so we get a yearly average for Jan - Dec. 
    5. Choose whichever start and end year you'd like. I'd recommend downloading the full range of data. 
    6. Uncheck the "Display Base Period" box to the right of the drop-down menus. 
    7. On the website, click "Plot" to visualize the data you've chosen. Does it look like what you want?
    8. Below the plot, there's an option to Download the data. Download it as a CSV. 
9. **(***If using Binder***) Return to your Binder window and:**
    1. Click the "Upload Files" button just below the "Run" tab. It looks like an up arrow with a line underneath. Upload the csv file you just downloaded in step 1, and make sure it appears in your list of files.
    2. Then, right-click the csv file and click the option "Rename." Change the name of the csv file to "climate_data.csv"
9. **(***If not using Binder***) Return to the program you're using to run the notebook and:** 
    1. Save the csv file you just downloaded in the same directory as your Jupyter notebook. 
    2. Change the name of your csv file to "climate_data.csv"
10. **You may begin the notebook!**

DISCLAIMER: if you choose a region with missing data points, the program will not calculate the long-term linear trendline. 

### Step 1: Import Python modules
This section of code won't produce any output. The "import" statements below make sure the code throughout this entire notebook can be executed correctly. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import datetime

### Step 2: Read and format the csv file to prepare it for analysis. 
This section allows you to visualize the first few rows of your dataset and make sure it's been read in correctly. If the code below produces an error, there may be an issue with the csv file. 

In [ ]:
# Getting the geographical area from the csv.
name = pd.read_csv("climate_data.csv", usecols = [0])
if (name.columns[0])[-5:] == "ature": 
    region = (name.columns[0])[:-37]
else: #if the "region name" cell doesn't include that extra info, all is good!
    region = name.columns[0]
    
# Getting the variable from the csv.
var = pd.read_csv("climate_data.csv", usecols = [1])
place = (var.columns[0]).find('December')
variable = (var.columns[0])[(place + 9):]

# Reading in the csv file. 
remove_header_lines = [0,1,2] # lines 0, 1, and 2 are not data, and we need to remove them before opening the CSV
climate_data = pd.read_csv("climate_data.csv", skiprows = lambda x: x in remove_header_lines)

# Now let's change the "Date" column from a string to datetime. This way, Python knows we're working with dates and will organize them chronologically during plotting.
climate_data["Date"] = pd.to_datetime(climate_data["Date"], format='%Y%m') 

# We'll also change the date to "Year" only, because this data represents an annual average. 
climate_data["Date"] = pd.DatetimeIndex(climate_data["Date"]).year

# If there's any missing data, the Value column should read -99. Since that skews the data, we'll change it to nan.
climate_data = climate_data.replace(-99, np.nan)

# Let's see a preview of what's inside. 
print(climate_data.head())

### Step 3: Calculate statistics from the dataset
This section is meant to understand the breadth and extent of the dataset. The output allows you to learn specific, quantitative facts about the climate data in your chosen location. Although many of these statistics will be plotted later, you may find it interesting to have written facts. 

In [ ]:
# Exploring the beginning and end of the dataset. 
data_start = (climate_data["Date"])[0] #here's the first row in our file.
data_end = (climate_data["Date"])[len(climate_data)-1] #and the last row in our file!
print(f"This {region} dataset begins in {data_start} and ends in {data_end}.")

# Hot & cold years on record 
hottest = climate_data.loc[climate_data['Value'].idxmax()] #maximum temperature in the dataset
coolest = climate_data.loc[climate_data['Value'].idxmin()] #minimum temperature in the dataset

print() #empty line for better formatting of the print statements.
print(f"The hottest year on record is {round(hottest.Date)} with an annual {variable} of {hottest.Value}\xb0F") #\xb0 is the degree symbol
print(f"The coolest year on record is {round(coolest.Date)} with an annual {variable} of {coolest.Value}\xb0F")

# Ranking the most recent year as the nth hottest on record.
climate_data["Rank"] = climate_data["Value"].rank(axis=0, method='max', ascending=False)
recent_rank = ((climate_data["Rank"])[len(climate_data)-1])

print()
print(f"The most recent year in your dataset, {data_end}, ranks number {int(recent_rank)} out of {len(climate_data)} in the hottest years on record.")

# Rate of warming since beginning of record and since 1981 
z1 = np.polyfit(climate_data["Date"], climate_data['Value'], 1) #making a linear trend line with all data!
p1 = np.poly1d(z1)

index_of_1981 = int(climate_data[climate_data['Date']==1981].index[0]) #finding the location of the 1981 data so we can refine our dataset 
z2 = np.polyfit((climate_data["Date"])[index_of_1981:], (climate_data['Value'])[index_of_1981:], 1) #limiting the dataset to 1981-most recent, then making a trendline
p2 = np.poly1d(z2)

print()
print(f"The annual {variable} has been increasing at a rate of {round((z1[0])*10,2)}\xb0F per decade since {data_start}")
print(f"The annual {variable} has been increasing at a rate of {round((z2[0])*10,2)}\xb0F per decade since 1981")

### Step 4: Plot all temperature data, including the range of temperatures. 
This section allows you to visualize the hottest and coldest recorded years for your location and understand how the annual temperatures change from year to year.

In [ ]:
# Plotting
fix, ax = plt.subplots() 
ax.plot(climate_data["Date"], climate_data['Value'], color="black")
plt.xlabel("Year")
plt.grid()
plt.ylabel(f"Annual {variable} (\xb0F)")
plt.title(f"Annual {variable}: {region} ({str(data_start)} to {str(data_end)})") 

# Putting the hottest/coldest year stats on the plot
plt.axhline(y=hottest.Value,color="red",linestyle = 'dashed', label = f"Hottest year ({hottest.Value}\xb0F)")
plt.axhline(y=coolest.Value,color="blue",linestyle = 'dashed', label = f"Coolest year ({coolest.Value}\xb0F)")
plt.legend()

As you can see from the above plot, the annual temperature changes a lot from year-to-year, sometimes increasing and sometimes decreasing. Keep in mind that there are numerous meteorological variables that affect the temperature of a region, and that large variability over the course of a couple years is normal and expected. 

When scientists analyze temperatures in the context of the climate, there are a couple of strageties we use to look past the year-to-year variability and see the larger picture.

### Step 5: Plot a rolling average to visualize general trends
This section uses a rolling average to visualize temperature data. The "rolling average" calculation in this notebook smooths the year-to-year variability shown in the previous section so the temperature trends become more apparent. In the graph below, each datapoint is an average of the previous 5 years. 

In [ ]:
# Plot
fix, ax = plt.subplots()
ax.plot(climate_data["Date"], climate_data['Value'].rolling(5).mean(), color="black")
plt.xlabel("Year")
plt.grid()
plt.ylabel(f"Annual {variable} (\xb0F), \n averaged over 5 years") 
plt.title(f"Rolling mean annual {variable}: \n{region} ({str(data_start)} to {str(data_end)})") 

This plot contains the same data and information as the first one, but the year-to-year variation caused by meteorological variation has mostly disappeared. Now, the larger-scale temperature trends are more easily visualized.

### Step 6: Plotting linear trend lines (across the whole record and since 1981)
This section also analyzes trends in the temperature data by calculating linear trend line values. Using the annual temperature data located in the csv file, this code calculates a trend over the entire recorded period, then a trend since 1981 (a common start date for climate trend calculations, as global temperatures began to show a notable increase at this time).

In [ ]:
# Let's make a linear trend line!
fix, ax = plt.subplots()
ax.plot(climate_data["Date"], climate_data['Value'], color="black")
plt.xlabel("Year")
plt.grid()
plt.ylabel(f"Annual {variable} (\xb0F)") 
plt.title(f"Annual {variable} trends in {region}")

#add trendline to plot (entire range of dates)
plt.plot(climate_data["Date"], p1(climate_data["Date"]), color = "darkturquoise", label = f"Linear trendline: {data_start} to {data_end}", linewidth=3) # we already calculated the trendline above, so now we just have to plot it!
plt.legend()

#add trendline to plot (since 1981)
plt.plot((climate_data["Date"])[index_of_1981:], (p2(climate_data["Date"]))[index_of_1981:], color = "orange", label = f"Linear trendline: 1981 to {data_end}", linewidth=3) # we already calculated the trendline above, so now we just have to plot it!
plt.legend()

print(f"Equation for {data_start} to {data_end} trendline: {p1}")
print(f"Equation for 1981 to {data_end} trendline: {p2}")

It is likely that the above plot shows a mild temperature increase across the entire record, and a steep increase since 1981. If this is the case, you can see how warming has accelerated in the more recent years, whereas warming trends were more mild at the beginning of the record (if your dataset extends back to the early 1900s). 

### Step 7: Plot temperature anomalies across the whole record
The visualization of temperature anomalies is another way that scientists analyze temperature behavior under climate change. In this scenario, an anomaly is the difference between the average temperature (calculated for the 20th century) and the annual temperature value from each year. The plot produced will show temperature anomalies, or the deviation from "normal" temperatures, for your selected region. 

In [ ]:
# Another way to visualize climate data is through anomalies: anomalies show how far each data point is from the mean value.
# Our mean value will be the average temperature from the 20th century (1901-2000). Let's print that out.
try: #this makes sure your data is plotted, even if it doesn't begin in 1901. We'll just average from the start of your dataset.
    index_of_1901 = int(climate_data[climate_data['Date']==1901].index[0])
except:
    index_of_1901 = int(climate_data[climate_data['Date']==data_start].index[0])
index_of_2001 = int(climate_data[climate_data['Date']==2001].index[0]) #Python is silly, so we have to include one year beyond our target range.
mean_dataset = (climate_data["Value"])[index_of_1901:index_of_2001] #creating a new dataset that only contains our 1901-2000 range
print(f"The annual {variable} in {region} during the 20th century was {round(mean_dataset.mean(),2)}\xb0F")

# Now let's make an anomaly plot. To do so, we'll need to calculate each year's difference from the mean temperature.
climate_data["Anomaly"] = climate_data["Value"] - mean_dataset.mean() #this calculation creates a new Pandas column!
print()
print(climate_data.head()) #let's look at the first few lines to understand what the "anomaly" column means.

# Alright, let's get to plotting! However, to format our colors, we'll first need to do some data reformatting. 
x = climate_data["Date"].to_numpy() #changing the data format to an array instead of a dataframe. Just something you have to do to perform certain functions.
y = climate_data["Anomaly"].to_numpy()

mask1 = y > 0 #if the anomaly is above zero, apply mask1. This lets us change the plot color to red down in the ax.bar command. 
mask2 = y <= 0 #if the anomaly is below zero, apply mask2. This lets us change the plot color to blue down in the ax.bar command. 

fix, ax = plt.subplots()
ax.bar(x[mask1], y[mask1], color = 'firebrick') #let's do some fun colors! Reddish if above 0, blueish if below.
ax.bar(x[mask2], y[mask2], color = 'royalblue')
plt.xlabel("Year")
plt.grid()
plt.ylabel("Difference from 1901 - 2000 average (\xb0F)") 
plt.title(f"Annual {variable} in {region}")

In this plot, the red bars show where the annual temperature was abnormally high, and the blue bars show where the annual temperature was abnormally low. The taller the bar, the further the temperature was from average. 

References and continued reading:
https://www.climate.gov/news-features/understanding-climate/climate-change-global-temperature